In [ ]:
import os

# Set the target folder
yolox_dir = "./YOLOX"
os.chdir(yolox_dir)
print(f"⚠️ Changed working directory to: {os.getcwd()}")

In [ ]:
import os
import glob
import subprocess

def setup_complete_vs_environment():
    """
    Complete Visual Studio environment setup for PyTorch C++ extensions
    """
    vs_path = r"C:\Program Files (x86)\Microsoft Visual Studio\2019\BuildTools"
    
    print("Setting up complete Visual Studio environment...")
    
    # 1. Find Windows SDK
    sdk_bases = [
        r"C:\Program Files (x86)\Windows Kits\10",
        r"C:\Program Files\Windows Kits\10"
    ]
    
    include_paths = []
    lib_paths = []
    
    for base in sdk_bases:
        if os.path.exists(base):
            # Find SDK versions
            include_base = os.path.join(base, "Include")
            lib_base = os.path.join(base, "Lib")
            
            if os.path.exists(include_base):
                versions = [d for d in os.listdir(include_base) if d.startswith("10.")]
                if versions:
                    latest_version = max(versions)
                    sdk_include = os.path.join(include_base, latest_version)
                    sdk_lib = os.path.join(lib_base, latest_version)
                    
                    # Add SDK include paths
                    include_paths.extend([
                        os.path.join(sdk_include, "ucrt"),
                        os.path.join(sdk_include, "um"),
                        os.path.join(sdk_include, "shared"),
                        os.path.join(sdk_include, "winrt"),
                        os.path.join(sdk_include, "cppwinrt"),
                    ])
                    
                    # Add SDK library paths
                    lib_paths.extend([
                        os.path.join(sdk_lib, "ucrt", "x64"),
                        os.path.join(sdk_lib, "um", "x64"),
                    ])
                    
                    print(f"Found Windows SDK {latest_version}")
                    break
    
    # 2. Find MSVC tools
    vc_tools_path = os.path.join(vs_path, "VC", "Tools", "MSVC")
    if os.path.exists(vc_tools_path):
        versions = os.listdir(vc_tools_path)
        if versions:
            latest_version = max(versions)
            msvc_base = os.path.join(vc_tools_path, latest_version)
            
            # Add MSVC include paths
            include_paths.extend([
                os.path.join(msvc_base, "include"),
                os.path.join(msvc_base, "atlmfc", "include"),
            ])
            
            # Add MSVC library paths
            lib_paths.extend([
                os.path.join(msvc_base, "lib", "x64"),
                os.path.join(msvc_base, "atlmfc", "lib", "x64"),
            ])
            
            # Add compiler to PATH
            bin_path = os.path.join(msvc_base, "bin", "Hostx64", "x64")
            current_path = os.environ.get('PATH', '')
            os.environ['PATH'] = f"{bin_path};{current_path}"
            
            print(f"Found MSVC {latest_version}")
    
    # 3. Set environment variables
    # Include paths
    current_include = os.environ.get('INCLUDE', '')
    new_include = ';'.join(include_paths + ([current_include] if current_include else []))
    os.environ['INCLUDE'] = new_include
    
    # Library paths
    current_lib = os.environ.get('LIB', '')
    new_lib = ';'.join(lib_paths + ([current_lib] if current_lib else []))
    os.environ['LIB'] = new_lib
    
    # Essential VS environment variables
    os.environ.update({
        'DISTUTILS_USE_SDK': '1',
        'MSSdk': '1',
        'VS160COMNTOOLS': f"{vs_path}\\Common7\\Tools\\",
        'VCINSTALLDIR': f"{vs_path}\\VC\\",
        'WindowsSDKDir': sdk_bases[0] + "\\" if os.path.exists(sdk_bases[0]) else "",
        'PLATFORM': 'x64',
        'PROCESSOR_ARCHITECTURE': 'AMD64',
    })
    
    print(f"Set up {len(include_paths)} include paths")
    print(f"Set up {len(lib_paths)} library paths")
    
    # 4. Verify key libraries exist
    key_libs = ['kernel32.lib', 'msvcprt.lib', 'msvcrt.lib', 'oldnames.lib']
    found_libs = {}
    
    for lib_name in key_libs:
        for lib_path in lib_paths:
            lib_file = os.path.join(lib_path, lib_name)
            if os.path.exists(lib_file):
                found_libs[lib_name] = lib_file
                break
    
    print(f"\nFound libraries: {list(found_libs.keys())}")
    missing_libs = set(key_libs) - set(found_libs.keys())
    if missing_libs:
        print(f"Missing libraries: {list(missing_libs)}")
        
        # Try to find them in other locations
        print("Searching for missing libraries...")
        for lib_name in missing_libs:
            for lib_path in lib_paths:
                if os.path.exists(lib_path):
                    all_libs = [f for f in os.listdir(lib_path) if f.endswith('.lib')]
                    similar = [lib for lib in all_libs if lib_name.split('.')[0] in lib]
                    if similar:
                        print(f"  In {lib_path}: found similar {similar[:3]}")
    
    return len(missing_libs) == 0

def clean_torch_cache():
    """Clean PyTorch extension cache to force recompilation"""
    cache_path = os.path.expanduser("~/.cache/torch_extensions")
    if os.path.exists(cache_path):
        import shutil
        shutil.rmtree(cache_path)
        print("Cleaned PyTorch extensions cache")
    
    # Also clean the specific cache location
    local_cache = r"C:\Users\aarnaizl\AppData\Local\torch_extensions"
    if os.path.exists(local_cache):
        import shutil
        shutil.rmtree(local_cache)
        print("Cleaned local PyTorch extensions cache")

# Run the setup
print("=== Setting up Visual Studio Environment ===")
success = setup_complete_vs_environment()

if success:
    print("\n✅ Environment setup complete!")
    print("🗑️ Cleaning PyTorch cache for fresh compilation...")
    clean_torch_cache()
    print("\n🚀 Ready to run training with fast_cocoeval!")
else:
    print("\n⚠️ Some libraries are missing. You may need to:")
    print("Check Visual Studio Build Tools is installed with the required modules (see README.md)")
    
print("\nYou can now run your training command.")

In [ ]:
%matplotlib inline
import os
import json
import torch
from pprint import pprint
import numpy as np
from tqdm import tqdm
import time

# YOLOX specific imports
from yolox.exp import get_exp
from yolox.utils import adjust_status, ModelEMA, postprocess
from yolox.data import COCODataset, ValTransform
from torch.utils.data import DataLoader

# -------- CONFIG --------
# Use the exact same paths as training
data_dir = "D:/Ainhoa/traffic_signs_data/DFG_detection/dataset_coco_ready_original_yolox"
val_ann = "coco_val_annotations_remapped.json"  # Same as training
model_weights_path = r'D:\Ainhoa\traffic_signs_data\models\YOLOX_outputs_dfg_m\yolox_m\best_ckpt.pth'
output_dir = "yolox_eval"
class_names_path = "classes.txt"

# Load experiment configuration (same as training)
exp_file = "yolox_m.py"  # Your experiment file
exp = get_exp(exp_file, None)

# -------- LOAD CLASS NAMES --------
with open(class_names_path, "r") as f:
    class_names = [line.strip() for line in f if line.strip()]
num_classes = len(class_names)

print(f"[INFO] Using experiment: {exp.exp_name}")
print(f"[INFO] Test size: {exp.test_size}")
print(f"[INFO] Confidence threshold: {exp.test_conf}")
print(f"[INFO] NMS threshold: {exp.nmsthre}")
print(f"[INFO] Number of classes: {exp.num_classes}")

# -------- CONFIGURE AND LOAD MODEL (EXACTLY LIKE TRAINING) --------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = exp.get_model()

# Load checkpoint
ckpt = torch.load(model_weights_path, map_location="cpu")

# Load model weights (regular model first)
print("Loading regular model weights")
model.load_state_dict(ckpt["model"])

model.to(device)
model.eval()

# -------- CREATE EVALUATOR (EXACTLY LIKE TRAINING) --------
os.makedirs(output_dir, exist_ok=True)

# Create evaluator exactly like training
evaluator = exp.get_evaluator(
    batch_size=1,  # Use batch_size=1 for evaluation
    is_distributed=False
)

print("Using regular model for evaluation")
evalmodel = model

# -------- RUN EVALUATION (EXACTLY LIKE TRAINING) --------
print("[INFO] Running evaluation...")

# Adjust model status for evaluation (exactly like training)
with adjust_status(evalmodel, training=False):
    # Use the exact same evaluation method as training
    ap50_95, ap50, summary = exp.eval(
        evalmodel, evaluator, is_distributed=False
    )

# -------- PRINT RESULTS EXACTLY LIKE TRAINING --------
print("\n" + "="*50)
print("EVALUATION RESULTS")
print("="*50)
print(summary)

# -------- GENERATE COCO RESULTS JSON FOR PLOTTING --------
print("[INFO] Found val2017 subdirectory")
val_dataset = COCODataset(
        data_dir=data_dir,
        json_file=val_ann,
        name="val2017",  # Use val2017 subdirectory
        img_size=exp.test_size,
        preproc=ValTransform(legacy=False),
        cache=False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    pin_memory=False,
    drop_last=False,
)

# Collect all predictions in COCO format
coco_results = []
inference_time = 0
nms_time = 0
n_samples = 0

with torch.no_grad():
    for cur_iter, (imgs, _, info_imgs, ids) in enumerate(tqdm(val_loader, desc="Processing images")):
        imgs = imgs.to(device, non_blocking=True).float()
        
        # Get image info
        img_id = int(ids[0])
        img_h = float(info_imgs[0])
        img_w = float(info_imgs[1])
        
        # Inference (exactly like training)
        inf_start = time.time()
        outputs = evalmodel(imgs)
        inf_end = time.time()
        inference_time += inf_end - inf_start
        
        # Postprocessing (exactly like training)
        nms_start = time.time()
        outputs = postprocess(
            outputs, exp.num_classes, exp.test_conf, exp.nmsthre, class_agnostic=True
        )
        nms_end = time.time()
        nms_time += nms_end - nms_start
        
        # Process results
        if outputs[0] is not None:
            output = outputs[0].cpu()
            bboxes = output[:, 0:4]
            
            # Scale back to original image (exactly like training)
            scale = min(exp.test_size[0] / img_h, exp.test_size[1] / img_w)
            bboxes /= scale
            
            # Extract scores and class IDs
            obj_conf = output[:, 4]
            cls_conf = output[:, 5]
            scores = obj_conf * cls_conf  # Combined confidence
            cls_ids = output[:, 6]
            
            # Convert to COCO format
            for i in range(len(bboxes)):
                x1, y1, x2, y2 = bboxes[i]
                width = x2 - x1
                height = y2 - y1
                
                # Only save valid bboxes
                if width > 0 and height > 0:
                    coco_results.append({
                        "image_id": img_id,
                        "category_id": int(cls_ids[i]) + 1,  # COCO categories start from 1
                        "bbox": [float(x1), float(y1), float(width), float(height)],
                        "score": float(scores[i])
                    })
        
        n_samples += 1

# -------- SAVE RESULTS --------
# Save COCO results JSON
coco_results_file = os.path.join(output_dir, "coco_instances_results.json")
with open(coco_results_file, "w") as f:
    json.dump(coco_results, f)

print(f"\n[INFO] COCO results saved to: {coco_results_file}")
print(f"[INFO] Total detections: {len(coco_results)}")

# Save evaluation summary
results = {
    "summary": summary,
    "ap50_95": ap50_95,
    "ap50": ap50,
    "total_detections": len(coco_results),
    "avg_inference_time_ms": (inference_time / n_samples) * 1000,
    "avg_nms_time_ms": (nms_time / n_samples) * 1000,
}

results_file = os.path.join(output_dir, "training_style_evaluation.json")
with open(results_file, "w") as f:
    json.dump(results, f, indent=2)

print(f"[INFO] Evaluation summary saved to: {results_file}")

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve
from collections import defaultdict


class DetectionEvaluator:
    """Class to handle object detection evaluation with confusion matrices and PR curves."""
    
    def __init__(self, config):
        self.config = config
        self.class_names = self._load_class_names()
        self.num_classes = len(self.class_names)
        self.coco_gt, self.coco_pred = self._load_data()
        self.catid_to_index = self._build_category_mapping()
        
        os.makedirs(config['output_dir'], exist_ok=True)
    
    def _load_class_names(self):
        """Load class names from file."""
        with open(self.config['class_names_path'], "r") as f:
            return [line.strip() for line in f if line.strip()]
    
    def _load_data(self):
        """Load COCO ground truth and prediction data."""
        with open(self.config['coco_json'], 'r') as f:
            coco_gt = json.load(f)
        with open(self.config['prediction_json'], 'r') as f:
            coco_pred = json.load(f)
        return coco_gt, coco_pred
    
    def _build_category_mapping(self):
        """Build mapping from category ID to class index."""
        return {cat['id']: idx for idx, cat in enumerate(self.coco_gt['categories'])}
    
    def _organize_predictions_by_image(self):
        """Organize ground truth and predictions by image."""
        # Ground truth by image
        gt_by_image = {}
        for ann in self.coco_gt['annotations']:
            img_id = ann['image_id']
            class_idx = self.catid_to_index[ann['category_id']]
            gt_by_image.setdefault(img_id, []).append(class_idx)
        
        # Predictions by image (filtered by confidence)
        pred_by_image = {}
        min_conf = self.config['min_confidence_thresh']
        for pred in self.coco_pred:
            if pred["score"] >= min_conf:
                img_id = pred["image_id"]
                class_id = self.catid_to_index.get(pred["category_id"], -1)
                pred_by_image.setdefault(img_id, []).append(class_id)
        
        return gt_by_image, pred_by_image
    
    def _prepare_confusion_matrix_data(self, gt_by_image, pred_by_image):
        """Prepare data for confusion matrix calculation."""
        all_gts, all_preds = [], []
        
        for img_id, gts in gt_by_image.items():
            preds = pred_by_image.get(img_id, [])
            
            # Matched predictions
            for gt, pred in zip(gts, preds):
                all_gts.append(gt)
                all_preds.append(pred)
            
            # Unmatched ground truths (false negatives)
            for gt in gts[len(preds):]:
                all_gts.append(gt)
                all_preds.append(self.num_classes)  # unmatched class
            
            # Unmatched predictions (false positives)
            for pred in preds[len(gts):]:
                all_gts.append(self.num_classes)
                all_preds.append(pred)
        
        # Add missing class if needed
        if self.num_classes in all_preds or self.num_classes in all_gts:
            self.class_names.append("_missing_")
        
        return all_gts, all_preds
    
    def plot_confusion_matrices(self, all_gts, all_preds):
        """Plot confusion matrices (multiclass or binary aggregated)."""
        threshold = self.config['class_display_threshold']
        
        if self.num_classes <= threshold:
            self._plot_multiclass_confusion_matrices(all_gts, all_preds)
        else:
            self._plot_binary_confusion_matrix(all_gts, all_preds)
    
    def _plot_multiclass_confusion_matrices(self, all_gts, all_preds):
        """Plot standard multiclass confusion matrices."""
        labels = list(range(len(self.class_names)))
        configs = [
            (confusion_matrix(all_gts, all_preds, labels=labels), 
             "Confusion Matrix", "confusion_matrix.png"),
            (confusion_matrix(all_gts, all_preds, labels=labels, normalize="true"), 
             "Normalized Confusion Matrix", "confusion_matrix_normalized.png")
        ]
        
        for matrix, title, filename in configs:
            self._save_confusion_matrix(matrix, title, filename, figsize=(12, 10))
    
    def _plot_binary_confusion_matrix(self, all_gts, all_preds):
        """Plot binary confusion matrix (object vs no-object)."""
        y_true_binary = [1 if true < self.num_classes else 0 for true in all_gts]
        y_pred_binary = [1 if pred < self.num_classes else 0 for pred in all_preds]
        
        matrix = confusion_matrix(y_true_binary, y_pred_binary, labels=[0, 1])
        self._save_confusion_matrix(
            matrix, 
            "Aggregated Confusion Matrix (Object vs No Object)", 
            "confusion_matrix_aggregated.png",
            display_labels=["No Object", "Object"],
            figsize=(6, 5)
        )
    
    def _save_confusion_matrix(self, matrix, title, filename, display_labels=None, figsize=(12, 10)):
        """Save confusion matrix plot."""
        fig, ax = plt.subplots(figsize=figsize)
        labels = display_labels or self.class_names
        disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=labels)
        disp.plot(ax=ax, cmap="Blues", xticks_rotation=90)
        plt.title(title)
        plt.tight_layout()
        plt.savefig(os.path.join(self.config['output_dir'], filename))
        plt.close()
    
    @staticmethod
    def calculate_iou(box1, box2):
        """Calculate IoU between two bounding boxes in [x,y,w,h] format."""
        x1, y1, w1, h1 = box1
        x2, y2, w2, h2 = box2
        
        # Calculate intersection
        x_left = max(x1, x2)
        y_top = max(y1, y2)
        x_right = min(x1 + w1, x2 + w2)
        y_bottom = min(y1 + h1, y2 + h2)
        
        if x_right < x_left or y_bottom < y_top:
            return 0.0
        
        intersection_area = (x_right - x_left) * (y_bottom - y_top)
        union_area = w1 * h1 + w2 * h2 - intersection_area
        
        return intersection_area / union_area if union_area > 0 else 0.0
    
    def build_pr_curve_data(self, iou_threshold=0.5):
        """Build proper PR curve data for object detection with IoU matching."""
        # Organize ground truth by image and class with bounding boxes
        gt_by_image_class = defaultdict(list)
        gt_count_per_class = [0] * self.num_classes
        
        for ann in self.coco_gt['annotations']:
            img_id = ann['image_id']
            class_id = self.catid_to_index.get(ann['category_id'])
            if class_id is not None and class_id < self.num_classes:
                bbox = ann['bbox']
                gt_by_image_class[(img_id, class_id)].append({
                    'bbox': bbox,
                    'matched': False,
                    'ann_id': ann.get('id', len(gt_by_image_class))
                })
                gt_count_per_class[class_id] += 1
        
        # Sort all predictions by confidence (descending)
        sorted_predictions = sorted(self.coco_pred, key=lambda x: x['score'], reverse=True)
        
        # Initialize arrays for each class
        all_scores = [[] for _ in range(self.num_classes)]
        all_true = [[] for _ in range(self.num_classes)]
        
        # Process each prediction in order of decreasing confidence
        for pred in sorted_predictions:
            class_id = self.catid_to_index.get(pred['category_id'])
            if class_id is None or class_id >= self.num_classes:
                continue
                
            img_id = pred['image_id']
            pred_bbox = pred['bbox']
            confidence = pred['score']
            
            # Find ground truth objects of the same class in the same image
            gt_objects = gt_by_image_class.get((img_id, class_id), [])
            
            # Find the best matching ground truth (highest IoU)
            best_iou = 0
            best_gt_idx = -1
            
            for gt_idx, gt_obj in enumerate(gt_objects):
                if not gt_obj['matched']:  # Only consider unmatched ground truths
                    iou = self.calculate_iou(pred_bbox, gt_obj['bbox'])
                    if iou > best_iou:
                        best_iou = iou
                        best_gt_idx = gt_idx
            
            # Record the prediction
            all_scores[class_id].append(confidence)
            
            if best_iou >= iou_threshold and best_gt_idx >= 0:
                # True positive: mark GT as matched
                all_true[class_id].append(1)
                gt_objects[best_gt_idx]['matched'] = True
            else:
                # False positive
                all_true[class_id].append(0)
        
        return all_scores, all_true, gt_count_per_class
    
    def plot_pr_curves(self, all_scores, all_true, gt_count_per_class):
        """Plot all PR-related curves."""
        curve_configs = [
            ("PR", "PR_curve.png", "Recall", "Precision",
             lambda p, r, t: r, lambda p, r, t: p),
            ("F1", "F1_curve.png", "Confidence", "F1",
             lambda p, r, t: t, lambda p, r, t: (2 * p[1:] * r[1:] / (p[1:] + r[1:] + 1e-6))),
            ("Prec", "P_curve.png", "Confidence", "Precision",
             lambda p, r, t: t, lambda p, r, t: p[1:]),
            ("Recall", "R_curve.png", "Confidence", "Recall",
             lambda p, r, t: t, lambda p, r, t: r[1:])
        ]
        
        for config in curve_configs:
            self._plot_single_curve(config, all_scores, all_true, gt_count_per_class)
    
    def _plot_single_curve(self, curve_config, all_scores, all_true, gt_count_per_class):
        """Plot a single PR-related curve."""
        curve_type, filename, xlabel, ylabel, x_fn, y_fn = curve_config
        
        plt.figure(figsize=(10, 8))
        show_legend = self.num_classes <= self.config['class_display_threshold']
        
        # Plot individual class curves
        if show_legend:
            class_names_to_plot = (self.class_names[:-1] if "_missing_" in self.class_names 
                                 else self.class_names)
            
            for i, name in enumerate(class_names_to_plot):
                if self._should_skip_class(i, all_scores, gt_count_per_class):
                    continue
                
                try:
                    self._plot_class_curve(i, name, all_scores, all_true, x_fn, y_fn)
                except Exception as e:
                    print(f"Skipping class {name} due to error: {e}")
                    continue
        
        # Plot combined curve
        self._plot_combined_curve(all_scores, all_true, x_fn, y_fn)
        
        # Format and save plot
        self._format_and_save_plot(curve_type, filename, xlabel, ylabel)
    
    def _should_skip_class(self, class_idx, all_scores, gt_count_per_class):
        """Check if a class should be skipped in plotting."""
        return (len(all_scores[class_idx]) == 0 or 
                gt_count_per_class[class_idx] == 0 or 
                len(all_scores[class_idx]) < 3)
    
    def _plot_class_curve(self, class_idx, class_name, all_scores, all_true, x_fn, y_fn):
        """Plot curve for a single class."""
        p, r, t = precision_recall_curve(all_true[class_idx], all_scores[class_idx])
        
        if len(p) < 2 or len(r) < 2:
            return
            
        x, y = x_fn(p, r, t), y_fn(p, r, t)
        min_len = min(len(x), len(y))
        plt.plot(x[:min_len], y[:min_len], label=class_name, linewidth=1)
    
    def _plot_combined_curve(self, all_scores, all_true, x_fn, y_fn):
        """Plot combined curve for all classes."""
        try:
            all_combined_true = []
            all_combined_scores = []
            
            for i in range(self.num_classes):
                all_combined_true.extend(all_true[i])
                all_combined_scores.extend(all_scores[i])
            
            if len(all_combined_true) > 0 and len(set(all_combined_true)) > 1:
                p_c, r_c, t_c = precision_recall_curve(all_combined_true, all_combined_scores)
                x_c, y_c = x_fn(p_c, r_c, t_c), y_fn(p_c, r_c, t_c)
                min_len = min(len(x_c), len(y_c))
                plt.plot(x_c[:min_len], y_c[:min_len], 
                        label="all classes", linewidth=3, color='blue')
        except Exception as e:
            print(f"Error plotting combined curve: {e}")
    
    def _format_and_save_plot(self, curve_type, filename, xlabel, ylabel):
        """Format and save the plot."""
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.title(filename[:-4])
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        plt.grid(True)
        
        legend_loc = "lower right" if curve_type == 'Prec' else "lower left"
        plt.legend(fontsize='small', loc=legend_loc)
        plt.tight_layout()
        plt.savefig(os.path.join(self.config['output_dir'], filename))
        plt.close()
    
    def run_evaluation(self):
        """Run the complete evaluation pipeline."""
        print("Starting evaluation...")
        
        # Confusion matrices
        print("Generating confusion matrices...")
        gt_by_image, pred_by_image = self._organize_predictions_by_image()
        all_gts, all_preds = self._prepare_confusion_matrix_data(gt_by_image, pred_by_image)
        self.plot_confusion_matrices(all_gts, all_preds)
        
        # PR curves
        print("Generating PR curves...")
        all_scores, all_true, gt_count_per_class = self.build_pr_curve_data()
        self.plot_pr_curves(all_scores, all_true, gt_count_per_class)
        
        print(f"Evaluation complete! Results saved to {self.config['output_dir']}")

config = {
    "prediction_json": "yolox_eval/coco_instances_results.json",
    "coco_json": "D:/Ainhoa/traffic_signs_data/DFG_detection/dataset_coco_ready_original_yolox/annotations/coco_val_annotations_remapped.json",
    "class_names_path": "classes.txt",
    "output_dir": "yolox_eval",
    "class_display_threshold": 25,
    "min_confidence_thresh": 0.5
}

# Run evaluation
evaluator = DetectionEvaluator(config)
evaluator.run_evaluation()